# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('output_data/cities.csv')
del weather_df['Unnamed: 0']
weather_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Jamestown,42.0970,-79.2353,45.00,75,75,21.85,US,1619797843
1,Hobart,-42.8794,147.3294,51.80,82,2,5.75,AU,1619797935
2,Punta Arenas,-53.1500,-70.9167,41.00,93,75,9.22,CL,1619798059
3,Albany,42.6001,-73.9662,55.00,69,100,1.99,US,1619798060
4,Ribeira Grande,38.5167,-28.7000,62.60,63,40,8.05,PT,1619798062
5,Dikson,73.5069,80.5464,8.80,95,89,8.81,RU,1619798063
6,Fortuna,40.5982,-124.1573,55.99,100,1,5.75,US,1619797737
7,Marion,39.7684,-86.1555,66.99,45,20,20.71,US,1619797996
8,Rikitea,-23.1203,-134.9692,78.80,74,49,19.10,PF,1619797679
9,Bengkulu,-3.8004,102.2655,77.67,87,100,5.48,ID,1619797531


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_conditions = weather_df.loc[(weather_df["Max Temp"] >= 75)&
                                  (weather_df["Cloudiness"] == 0) & 
                                  (weather_df['Humidity'] <= 40)].dropna()

ideal_conditions

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,Moussoro,13.6408,16.4901,103.98,4,0,7.11,TD,1619798064
41,Kumul,42.8000,93.4500,76.57,12,0,3.31,CN,1619798078
179,Ürümqi,43.8010,87.6005,77.00,15,0,8.95,CN,1619797931
207,Ýpsonas,34.6884,32.9574,77.00,33,0,19.57,CY,1619798163
235,Kidal,18.4411,1.4078,107.22,5,0,11.36,ML,1619798178
237,Buraidah,26.3260,43.9750,93.20,18,0,10.36,SA,1619798179
294,Minturno,41.2576,13.7237,78.80,24,0,3.71,IT,1619798214
360,Maţāy,28.4190,30.7792,100.31,6,0,19.71,EG,1619798245
380,Catalina Foothills,32.2979,-110.9187,79.00,22,0,10.00,US,1619798260
386,Idlib,35.9306,36.6339,78.80,36,0,19.57,SY,1619798267


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_conditions

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
        
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,Moussoro,13.6408,16.4901,103.98,4,0,7.11,TD,1619798064,NaN
41,Kumul,42.8000,93.4500,76.57,12,0,3.31,CN,1619798078,Tianma Hotel
179,Ürümqi,43.8010,87.6005,77.00,15,0,8.95,CN,1619797931,JinJiang International Hotel Urumqi
207,Ýpsonas,34.6884,32.9574,77.00,33,0,19.57,CY,1619798163,Club Aphrodite
235,Kidal,18.4411,1.4078,107.22,5,0,11.36,ML,1619798178,NaN


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))